In [3]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
import random

In [4]:
class UserInput(TypedDict):
    player_name: str
    guesses: List[int]
    attempts: int
    lower_bound: int
    upper_bound: int

class AgentState(UserInput):
    target: int

In [5]:
def setup_game(state: AgentState) -> AgentState:
    """Setup the number guessing game with a random target"""
    target = random.randint(1, 20)
    print(f"🎮 Game started for {state['player_name']}! Guess a number between 1 and 20.")
    return {
        **state,
        "target": target
    }


In [6]:
def make_guess(state: AgentState) -> AgentState:
    """Make a smart guess within the current bounds"""
    lower = state["lower_bound"]
    upper = state["upper_bound"]
    
    # Make a smart guess (middle of the range)
    guess = (lower + upper) // 2
    
    print(f"🤔 Guessing: {guess} (range: {lower}-{upper})")
    
    return {
        **state,
        "attempts": state["attempts"] + 1,
        "guesses": state["guesses"] + [guess]
    }


In [7]:
def provide_hint(state: AgentState) -> AgentState:
    """Provide hint after a guess and update bounds accordingly"""
    last_guess = state["guesses"][-1]
    target = state["target"]
    
    if last_guess < target:
        print(f"📈 Too low! Try higher than {last_guess}")
        return {
            **state,
            "lower_bound": last_guess + 1
        }
    elif last_guess > target:
        print(f"📉 Too high! Try lower than {last_guess}")
        return {
            **state,
            "upper_bound": last_guess - 1
        }
    else:
        print(f"🎉 Correct! The number was {target}")
        return state


In [8]:
def should_continue(state: AgentState) -> str:
    """Decide whether to continue guessing or end the game"""
    if not state["guesses"]:
        return "continue"
    
    last_guess = state["guesses"][-1]
    target = state["target"]
    max_attempts = 7
    
    # Win condition
    if last_guess == target:
        print(f"🏆 {state['player_name']} won in {state['attempts']} attempts!")
        return "end"
    
    # Max attempts reached
    if state["attempts"] >= max_attempts:
        print(f"💀 Game over! The number was {target}. Better luck next time!")
        return "end"
    
    # Continue guessing
    return "continue"


In [9]:
# Build the graph  
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("setup", setup_game)
graph.add_node("guess", make_guess)
graph.add_node("hint", provide_hint)

# Add edges
graph.add_edge(START, "setup")
graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint")

# Add conditional edge
graph.add_conditional_edges(
    "hint",
    should_continue,
    {
        "continue": "guess",
        "end": END
    }
)

# Compile the graph
app = graph.compile()


In [10]:
# Test the game with the exact input format from the exercise
user_input: UserInput = {
    "player_name": "Student", 
    "guesses": [],
    "attempts": 0,
    "lower_bound": 1,
    "upper_bound": 20
}

# Run the game - cast to AgentState since setup node will add the target field
from typing import cast
result = app.invoke(cast(AgentState, user_input))
print(f"\n🎯 Final result: {result}")


🎮 Game started for Student! Guess a number between 1 and 20.
🤔 Guessing: 10 (range: 1-20)
📈 Too low! Try higher than 10
🤔 Guessing: 15 (range: 11-20)
📉 Too high! Try lower than 15
🤔 Guessing: 12 (range: 11-14)
📉 Too high! Try lower than 12
🤔 Guessing: 11 (range: 11-11)
🎉 Correct! The number was 11
🏆 Student won in 4 attempts!

🎯 Final result: {'player_name': 'Student', 'guesses': [10, 15, 12, 11], 'attempts': 4, 'lower_bound': 11, 'upper_bound': 11, 'target': 11}
